一般来说，具备高送转预期的个股，都具有总市值低、每股公积金高、每股收益大，流通股本少的特点。当然，也还有其它的因素，比如当前股价、经营收益变动情况、以及以往分红送股习惯等等。

这里我们暂时只考虑每股公积金、每股收益、流通股本和总市值四个因素，将公积金大于等于5元，每股收益大于等于5毛，流通股本在3亿以下，总市值在100亿以内作为高送转预期目标（这些参数大家可根据自己的经验随意调整）。

In [4]:
import tushare as ts

In [6]:
#调取基本面数据
basic =  ts.get_stock_basics()
#行情和市值数据
hq = ts.get_today_all()

[Getting data:]#####################################

In [11]:
#数据清理
#如果停牌，将当前股价定义为上一个交易日股价
hq['trade'] = hq.apply(lambda x:x.settlement if x.trade==0 else x.trade, axis=1)

In [15]:
#只选取流通股股本，总股本， 每股公积金，每股收益
basedata = basic[['outstanding', 'totals', 'reservedPerShare', 'esp']]
#只选取股票代码, 名称，当前价格，总市值，流通市值
hqdata = hq[['code', 'name', 'trade', 'mktcap', 'nmc']]
#将行情数据的code设为index
hqdata = hqdata.set_index('code')

#合并两个数据表（目前index都是code了）
data = basedata.merge(hqdata, left_index=True, right_index=True)


In [17]:
#将市值换算成亿
data['mktcap'] = data['mktcap'] / 10000
data['nmc'] = data['nmc'] / 10000

In [22]:
#公积金大于5 
res = data.reservedPerShare >= 5
#流通股本小于3亿股
out = data.outstanding <= 30000
#每股收益大于5毛
eps = data.esp >= 0.5
#总市值小于100亿
mktcap = data.mktcap <= 100

In [23]:
#取并集
allcrit = res & out & eps & mktcap

In [26]:
#调取结果
selected = data[allcrit]
selected

,outstanding,totals,reservedPerShare,esp,name,trade,mktcap,nmc
code,,,,,,,,
600419,8332.04,10355.71,5.86,0.721,天润乳业,57.99,60.052825,48.317487
300547,1495.00,5977.91,5.36,1.210,川环科技,114.34,68.351554,17.093830
300543,1500.00,6000.00,5.50,1.360,朗科智能,137.47,82.482000,20.620500
300535,1494.00,5974.00,5.26,0.840,达威股份,103.47,61.812978,15.458418
300507,1667.00,6667.00,5.36,1.130,苏奥传感,136.84,91.231228,22.811228
300501,1338.00,5338.00,5.99,0.950,海顺新材,130.02,69.404676,17.396676
300500,1500.00,6150.00,7.07,0.660,苏州设计,111.99,68.873850,16.798500
300473,4674.83,10000.00,9.92,0.897,德尔股份,78.12,78.120000,36.519811
300471,8693.78,14830.79,6.45,0.603,厚普股份,48.93,72.567104,42.538861


In [33]:
selected.to_excel(u'高转送.xls')